In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

## File constants

In [2]:
targetWidth = 1024
targetHeight = 512

In [3]:
from collections import namedtuple

# Based on https://github.com/mcordts/cityscapesScripts
CityscapesClass = namedtuple('CityscapesClass', ['name', 'id', 'train_id', 'category', 'category_id',
                                                 'has_instances', 'ignore_in_eval', 'color'])
classes = [
    CityscapesClass('unlabeled',            0, 19, 'void', 0, False, True, (0, 0, 0)),
    CityscapesClass('ego vehicle',          1, 19, 'void', 0, False, True, (0, 0, 0)),
    CityscapesClass('rectification border', 2, 19, 'void', 0, False, True, (0, 0, 0)),
    CityscapesClass('out of roi',           3, 19, 'void', 0, False, True, (0, 0, 0)),
    CityscapesClass('static',               4, 19, 'void', 0, False, True, (0, 0, 0)),
    CityscapesClass('dynamic',              5, 19, 'void', 0, False, True, (111, 74, 0)),
    CityscapesClass('ground',               6, 19, 'void', 0, False, True, (81, 0, 81)),
    CityscapesClass('road',                 7, 0, 'flat', 1, False, False, (128, 64, 128)),
    CityscapesClass('sidewalk',             8, 1, 'flat', 1, False, False, (244, 35, 232)),
    CityscapesClass('parking',              9, 19, 'flat', 1, False, True, (250, 170, 160)),
    CityscapesClass('rail track',           10, 19, 'flat', 1, False, True, (230, 150, 140)),
    CityscapesClass('building',             11, 2, 'construction', 2, False, False, (70, 70, 70)),
    CityscapesClass('wall',                 12, 3, 'construction', 2, False, False, (102, 102, 156)),
    CityscapesClass('fence',                13, 4, 'construction', 2, False, False, (190, 153, 153)),
    CityscapesClass('guard rail',           14, 19, 'construction', 2, False, True, (180, 165, 180)),
    CityscapesClass('bridge',               15, 19, 'construction', 2, False, True, (150, 100, 100)),
    CityscapesClass('tunnel',               16, 19, 'construction', 2, False, True, (150, 120, 90)),
    CityscapesClass('pole',                 17, 5, 'object', 3, False, False, (153, 153, 153)),
    CityscapesClass('polegroup',            18, 19, 'object', 3, False, True, (153, 153, 153)),
    CityscapesClass('traffic light',        19, 6, 'object', 3, False, False, (250, 170, 30)),
    CityscapesClass('traffic sign',         20, 7, 'object', 3, False, False, (220, 220, 0)),
    CityscapesClass('vegetation',           21, 8, 'nature', 4, False, False, (107, 142, 35)),
    CityscapesClass('terrain',              22, 9, 'nature', 4, False, False, (152, 251, 152)),
    CityscapesClass('sky',                  23, 10, 'sky', 5, False, False, (70, 130, 180)),
    CityscapesClass('person',               24, 11, 'human', 6, True, False, (220, 20, 60)),
    CityscapesClass('rider',                25, 12, 'human', 6, True, False, (255, 0, 0)),
    CityscapesClass('car',                  26, 13, 'vehicle', 7, True, False, (0, 0, 142)),
    CityscapesClass('truck',                27, 14, 'vehicle', 7, True, False, (0, 0, 70)),
    CityscapesClass('bus',                  28, 15, 'vehicle', 7, True, False, (0, 60, 100)),
    CityscapesClass('caravan',              29, 19, 'vehicle', 7, True, True, (0, 0, 90)),
    CityscapesClass('trailer',              30, 19, 'vehicle', 7, True, True, (0, 0, 110)),
    CityscapesClass('train',                31, 16, 'vehicle', 7, True, False, (0, 80, 100)),
    CityscapesClass('motorcycle',           32, 17, 'vehicle', 7, True, False, (0, 0, 230)),
    CityscapesClass('bicycle',              33, 18, 'vehicle', 7, True, False, (119, 11, 32)),
    CityscapesClass('license plate',        -1, 19, 'vehicle', 7, False, True, (0, 0, 142)),
]

train_id_to_color = [c.color for c in classes if (c.train_id != -1 and c.train_id != 19)]
train_id_to_color.append([0, 0, 0])
train_id_to_color = np.array(train_id_to_color)
id_to_train_id = np.array([c.train_id for c in classes])

## Helper functions

In [4]:
def createDir(dirPath):
    if(not os.path.isdir(dirPath)):
        os.mkdir(dirPath)

In [5]:
def createResizedDataSet(subFolder : str, dirs : str):
    imgCount = 0
    
    # create output directories for RGB images and labels
    outputRgbSubFolder = os.path.join(outputRgbDir, *[subFolder])
    createDir(outputRgbSubFolder)
    outputLabelSubFolder = os.path.join(outputLabelDir, *[subFolder])
    createDir(outputLabelSubFolder)
    
    # iterate through list of input directories
    for folderIndex, folder in enumerate(dirs):
        print(f'{folderIndex}. Reading {folder} data ...')
        
        # get files in the subdirectory
        rgbImgFolderPath = os.path.join(inputRgbDir, *[subFolder, folder])
        inputFiles = sorted(os.listdir(rgbImgFolderPath))
        
        for file in tqdm(inputFiles, total=len(inputFiles), position=0, leave=True):
            fileName = file.replace('_leftImg8bit.png','')
            outputFileName = f"{fileName}.png"
            
            # read input image, resize and write
            rgbImgPath = os.path.join(rgbImgFolderPath, *[file])
            rgbImg = cv2.imread(rgbImgPath)
            resizedRgbImg = cv2.resize(rgbImg, (targetWidth, targetHeight),
                     interpolation=cv2.INTER_NEAREST)
            cv2.imwrite(os.path.join(outputRgbDir, *[subFolder, outputFileName]), resizedRgbImg)

            # read label image, resize, encode to train id and write
            labelImgPath = os.path.join(inputLabelDir, *[subFolder, folder, f"{fileName}_gtFine_labelIds.png"])
            labelImg = cv2.imread(labelImgPath, -1)
            resizedLabelImg = cv2.resize(labelImg, (targetWidth, targetHeight),
                     interpolation=cv2.INTER_NEAREST)
            encodedResizedLabelImg = id_to_train_id[resizedLabelImg]
            cv2.imwrite(os.path.join(outputLabelDir, *[subFolder, outputFileName]), encodedResizedLabelImg)

            imgCount = imgCount + 1

    print(f"Resized {imgCount} images in {len(dirs)} folders")

In [6]:
def createResizedDemoVideoDataset(inputRootDir, dirs, outputRootDir):
    imgCount = 0
    
    # iterate through list of input directories    
    for folderIndex, folder in enumerate(dirs):
        print(f'{folderIndex}. Reading {folder} data ...')
        
        # get files in the subdirectory        
        rgbImgFolderPath = os.path.join(inputRootDir, *[folder])        
        inputFiles = sorted(os.listdir(rgbImgFolderPath))
        
        # create output directory
        outputRgbFolder = os.path.join(outputRootDir, *[folder])
        createDir(outputRgbFolder)
    
        for file in tqdm(inputFiles, total=len(inputFiles), position=0, leave=True):
            fileName = file.replace('_leftImg8bit.png','')
            outputFileName = f"{fileName}.png"
            
            # read input image, resize and write
            rgbImgPath = os.path.join(rgbImgFolderPath, *[file])
            rgbImg = cv2.imread(rgbImgPath)
            resizedRgbImg = cv2.resize(rgbImg, (targetWidth, targetHeight),
                     interpolation=cv2.INTER_NEAREST)
            cv2.imwrite(os.path.join(outputRgbFolder, *[outputFileName]), resizedRgbImg)
            
            imgCount = imgCount + 1
            
    print(f"Resized {imgCount} images in {len(dirs)} folders")            

## Convert Train, Validation folder

In [7]:
train_dirs = sorted(["jena", "zurich", "weimar", "ulm", "tubingen", "stuttgart",
              "strasbourg", "monchengladbach", "krefeld", "hanover",
              "hamburg", "erfurt", "dusseldorf", "darmstadt", "cologne",
              "bremen", "bochum", "aachen"])
val_dirs = sorted(["frankfurt", "munster", "lindau"])
test_dirs = sorted(["berlin", "bielefeld", "bonn", "leverkusen", "mainz", "munich"])

In [8]:
inputRgbDir = '../input/cityscapes/Cityspaces/images'
inputLabelDir = '../input/cityscapes/Cityspaces/gtFine'
outputRgbDir = 'leftImg8bit'
outputLabelDir = 'gtFine'

In [9]:
createDir(outputRgbDir)
createDir(outputLabelDir)

In [10]:
createResizedDataSet(subFolder='train', dirs=train_dirs)

0. Reading aachen data ...


100%|██████████| 174/174 [00:29<00:00,  5.88it/s]


1. Reading bochum data ...


100%|██████████| 96/96 [00:16<00:00,  5.77it/s]


2. Reading bremen data ...


...

16. Reading weimar data ...


100%|██████████| 142/142 [00:24<00:00,  5.81it/s]


17. Reading zurich data ...


100%|██████████| 122/122 [00:20<00:00,  5.88it/s]

Resized 2975 images in 18 folders


In [11]:
createResizedDataSet(subFolder='val', dirs=val_dirs)

0. Reading frankfurt data ...


100%|██████████| 267/267 [00:45<00:00,  5.81it/s]


1. Reading lindau data ...


100%|██████████| 59/59 [00:10<00:00,  5.78it/s]


2. Reading munster data ...


100%|██████████| 174/174 [00:29<00:00,  5.87it/s]

Resized 500 images in 3 folders


## Convert demoVideo folder

In [12]:
inputDemoVideoDir = '../input/cityscapes-demovideo/leftImg8bit/demoVideo'
outputDemoVideoDir = 'demoVideo'
demo_video_dirs = sorted(['stuttgart_00'])  # , 'stuttgart_01/', 'stuttgart_02/'

In [13]:
createDir(outputDemoVideoDir)

In [14]:
createResizedDemoVideoDataset(inputRootDir = inputDemoVideoDir, dirs = demo_video_dirs, 
                              outputRootDir = outputDemoVideoDir)

0. Reading stuttgart_00 data ...


100%|██████████| 599/599 [01:20<00:00,  7.44it/s]

Resized 599 images in 1 folders
